In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import datetime
import os

In [ ]:
!pip install kaggle

In [3]:
import os
os.environ['KAGGLE_API_TOKEN'] = 'KGAT_266314ff76d2a845638c14e51c9cc1dd'

In [6]:
!kaggle datasets download -d beekiran/sales-data-analysis

Dataset URL: https://www.kaggle.com/datasets/beekiran/sales-data-analysis
License(s): unknown
sales-data-analysis.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
import zipfile
zip_path = (r'C:\Users\devpr\Desktop\da projects\sales-data-analysis.zip')
extract_to = (r'C:\Users\devpr\Desktop\da projects\sales-data-analysis')

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
print(f'Files extarcted to: {extract_to}')

Files extarcted to: C:\Users\devpr\Desktop\da projects\sales-data-analysis


In [8]:
folder = (r'C:\Users\devpr\Desktop\da projects\sales-data-analysis')
print(os.listdir(folder))

['.ipynb_checkpoints', 'Sales Data.csv']


In [9]:
df = pd.read_csv(r'C:\Users\devpr\Desktop\da projects\sales-data-analysis\Sales Data.csv')
df.head(5)

,Unnamed: 0,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Sales,City,Hour
0,0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",12,1700.00,New York City,0
1,1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",12,600.00,New York City,7
2,2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",12,11.95,New York City,18
3,3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",12,149.99,San Francisco,15
4,4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,11.95,Atlanta,12


In [15]:
df.columns.tolist()

['Unnamed: 0',
 'Order ID',
 'Product',
 'Quantity Ordered',
 'Price Each',
 'Order Date',
 'Purchase Address',
 'Month',
 'Sales',
 'City',
 'Hour']

In [21]:
# Checking for Total Missing Values
print(f'Total Missing Values : {df.isna().sum().sum()}')

Total Missing Values : 0


In [93]:
#Checking Total Duplicates in two dimensions 1. Complete Transaction level, 2. Transactions with same order id, product, quantity with same timestamp

print(f'Total duplicate transactions : {df.duplicated().sum()}')

level2_dups = df[df[['Order ID', 'Product', 'Quantity Ordered', 'Order Date']].duplicated()]
true_dups = df.groupby(['Order ID', 'Product', 'Quantity Ordered', 'Order Date']).size().reset_index(name='count')
dups = len(true_dups[true_dups['count'] > 1])

print(f'\nTotal Duplicate Transactions with \nsame (order id, product, quantity, order date) = {dups}')

Total duplicate transactions : 0

Total Duplicate Transactions with 
same (order id, product, quantity, order date) = 264


In [95]:
# verifying data type
df.dtypes

#Order Date -> obj : to be transformed to datetime

Unnamed: 0            int64
Order ID              int64
Product              object
Quantity Ordered      int64
Price Each          float64
Order Date           object
Purchase Address     object
Month                 int64
Sales               float64
City                 object
Hour                  int64
dtype: object

In [103]:
price_consistent_checker = df.groupby('Product')['Price Each'].agg(['max','min','nunique','mean'])
price_consistent_checker[price_consistent_checker['nunique'] > 1]

,max,min,nunique,mean
Product,,,,


3.Data Preparation and Cleaning

In [117]:
df_new = df.copy()
df_new.columns

Index(['Unnamed: 0', 'Order ID', 'Product', 'Quantity Ordered', 'Price Each',
       'Order Date', 'Purchase Address', 'Month', 'Sales', 'City', 'Hour'],
      dtype='object')

In [119]:
#3.1 Renaming Column Names
df_new.columns = df_new.columns.str.lower().str.replace(' ', '_', regex = False)
df_new.columns

Index(['unnamed:_0', 'order_id', 'product', 'quantity_ordered', 'price_each',
       'order_date', 'purchase_address', 'month', 'sales', 'city', 'hour'],
      dtype='object')

In [123]:
df_new.head(5)

,unnamed:_0,order_id,product,quantity_ordered,price_each,order_date,purchase_address,month,sales,city,hour
0,0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",12,1700.00,New York City,0
1,1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",12,600.00,New York City,7
2,2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",12,11.95,New York City,18
3,3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",12,149.99,San Francisco,15
4,4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,11.95,Atlanta,12


In [126]:
#3.2 
#3.2.1 dropping columns and unnecessary duplicates
df_new = df_new.drop(columns = ['unnamed:_0'])

In [129]:
#dropped index
df_new = df_new.reset_index(drop = True)

In [ ]:
# 3.2.2 dropping duplicates of ['Order ID', 'Product', 'Quantity Ordered', 'Order Date']

#identifying duplicates
group_df = df_new.groupby(['order_id', 'product', 'quantity_ordered', 'order_date']).size().reset_index(name = 'count')
duplicates_df = group_df [group_df['count'] > 1]
duplicates_df

In [139]:
#droppig duplicates

df_new = df_new.drop_duplicates(subset = ['order_id', 'product', 'quantity_ordered', 'order_date'], keep='first')
df_new.shape
#duplicates successfully dropped

(185686, 10)

In [154]:
#Renaming Columns
new_column_names = {
    'quantity_ordered' : 'quantity',
    'price_each' : 'sell_price',
    'purchase_address' : 'address',
    'month' : 'month_num',
    'sales' : 'amount',
    'hour' :'order_hour_num',
    'order_date' : 'order_date_time'
    
}
df_new = df_new.rename(columns = new_column_names )
df_new.head()

,order_id,product,quantity,sell_price,order_date_time,address,month_num,amount,city,order_hour_num
0,295665,Macbook Pro Laptop,1,1700.00,2019-12-30 00:01:00,"136 Church St, New York City, NY 10001",12,1700.00,New York City,0
1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",12,600.00,New York City,7
2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",12,11.95,New York City,18
3,295668,27in FHD Monitor,1,149.99,2019-12-22 15:13:00,"410 6th St, San Francisco, CA 94016",12,149.99,San Francisco,15
4,295669,USB-C Charging Cable,1,11.95,2019-12-18 12:38:00,"43 Hill St, Atlanta, GA 30301",12,11.95,Atlanta,12


In [155]:
#3.3.1 order_date dtype correction

df_new['order_date_time'] = pd.to_datetime(df_new['order_date_time'])

In [160]:
df_new.dtypes

order_id                    int64
product                    object
quantity                    int64
sell_price                float64
order_date_time    datetime64[ns]
address                    object
month_num                   int64
amount                    float64
city                       object
order_hour_num              int64
dtype: object

In [163]:
# 3.4.1 creating new columns for time based analysis

df_new['month_name'] = df_new['order_date_time'].dt.month_name()
df_new['day_name'] = df_new['order_date_time'].dt.day_name()
df_new['quarter'] = df_new['order_date_time'].dt.quarter
df_new['day'] = df_new['order_date_time'].dt.day
df_new['date'] = df_new['order_date_time'].dt.date
df_new['time'] = df_new['order_date_time'].dt.time
df_new['weekday_num'] = df_new['order_date_time'].dt.weekday
df_new['week_num'] = df_new['order_date_time'].dt.isocalendar().week

In [167]:
# 3.4.2 sorted df_new as per date and time
df_new = df_new.sort_values(['date','time'], ascending = True)

In [180]:
#3.5 Categorizing data as per product category

df_new['category'] = 'Other'

def categorize_product(product):
    p = product.lower()
    if 'phone' in p:
        return 'Phone'
    if 'laptop' in p or 'macbook' in p or 'thinkpad' in p:
        return 'Laptop'
    if 'cable' in p or 'usb' in p or 'headphones' in p or 'airpods' in p:
        return 'Accessories'
    if 'batteries' in p:
        return 'Batteries'
    if 'monitor' in p:
        return 'Computers'
    if 'tv' in p or 'dryer' in p or 'washing' in p:
        return 'Appliances'
    return 'Others'


 
df_new['category']  = df_new['product'].apply(categorize_product)
df_new['category'].value_counts()

category
Phone          62099
Accessories    43469
Batteries      41170
Computers      23995
Laptop          8847
Appliances      6106
Name: count, dtype: int64

In [182]:
df_new.info()
#data is cleaned and prepared for EDA processes
df_new.head()

<class 'pandas.core.frame.DataFrame'>
Index: 185686 entries, 143110 to 5077
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   order_id         185686 non-null  int64         
 1   product          185686 non-null  object        
 2   quantity         185686 non-null  int64         
 3   sell_price       185686 non-null  float64       
 4   order_date_time  185686 non-null  datetime64[ns]
 5   address          185686 non-null  object        
 6   month_num        185686 non-null  int64         
 7   amount           185686 non-null  float64       
 8   city             185686 non-null  object        
 9   order_hour_num   185686 non-null  int64         
 10  month_name       185686 non-null  object        
 11  day_name         185686 non-null  object        
 12  quarter          185686 non-null  int32         
 13  day              185686 non-null  int32         
 14  date             18568

,order_id,product,quantity,sell_price,order_date_time,address,month_num,amount,city,order_hour_num,month_name,day_name,quarter,day,date,time,weekday_num,week_num,category
143110,147268,Wired Headphones,1,11.99,2019-01-01 03:07:00,"9 Lake St, New York City, NY 10001",1,11.99,New York City,3,January,Tuesday,1,1,2019-01-01,03:07:00,1,1,Phone
143917,148041,USB-C Charging Cable,1,11.95,2019-01-01 03:40:00,"760 Church St, San Francisco, CA 94016",1,11.95,San Francisco,3,January,Tuesday,1,1,2019-01-01,03:40:00,1,1,Accessories
145264,149343,Apple Airpods Headphones,1,150.00,2019-01-01 04:56:00,"735 5th St, New York City, NY 10001",1,150.00,New York City,4,January,Tuesday,1,1,2019-01-01,04:56:00,1,1,Phone
145915,149964,AAA Batteries (4-pack),1,2.99,2019-01-01 05:53:00,"75 Jackson St, Dallas, TX 75001",1,2.99,Dallas,5,January,Tuesday,1,1,2019-01-01,05:53:00,1,1,Batteries
145271,149350,USB-C Charging Cable,2,11.95,2019-01-01 06:03:00,"943 2nd St, Atlanta, GA 30301",1,23.90,Atlanta,6,January,Tuesday,1,1,2019-01-01,06:03:00,1,1,Accessories


In [189]:
#generate random profit and cost_price data as per industry standards as the data is not available for aalysis

np.random.seed(42)
df_new['profit_margin_pct'] = np.random.uniform(10,40, size = len(df_new)).round(2)

In [200]:
df_new['cost_price'] = (df_new['sell_price']*(1 - df_new['profit_margin_pct']/100)).round(2)
df_new['profit'] = (df_new['sell_price'] - df_new['cost_price'])*(df_new['quantity']).round(2)
df_new['profit'] = df_new['profit'].round(2)

In [201]:
# Save Data in CSV for further processing
df_new.to_csv('clean_sales_data.csv', index = False)